In [44]:
# import necessary libraries
import pandas as pd 
import numpy as np
import pickle
import torch
import math
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneGroupOut
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
import keras_tuner as kt
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import ccf

In [45]:
# load data
path = "datasets/CNAP_blood_pressure.csv"

df = pd.read_csv(path)
df.head()

,ecg_0,ecg_1,ecg_2,ecg_3,ecg_4,ecg_5,ecg_6,ecg_7,ecg_8,ecg_9,...,ppg_94,ppg_95,ppg_96,ppg_97,ppg_98,ppg_99,SBP,DBP,subject,trial
0,-784.0,-754.0,-732.0,-715.0,-689.0,-671.0,-658.0,-646.0,-638.0,-637.0,...,56603.0,56608.0,56612.0,56581.0,56551.0,56557.0,131.607056,58.044434,0,1.0
1,686.0,680.0,298.0,-294.0,-688.0,-596.0,-101.0,442.0,717.0,715.0,...,56247.0,56227.0,56203.0,56213.0,56224.0,56207.0,131.607056,58.044434,0,1.0
2,590.0,560.0,107.0,-534.0,-871.0,-632.0,-32.0,511.0,748.0,740.0,...,56374.0,56364.0,56351.0,56373.0,56399.0,56397.0,127.789307,57.366943,0,1.0
3,633.0,582.0,164.0,-412.0,-729.0,-529.0,17.0,523.0,724.0,691.0,...,56939.0,56922.0,56904.0,56922.0,56942.0,56930.0,128.906250,55.453491,0,1.0
4,694.0,424.0,-138.0,-671.0,-805.0,-463.0,119.0,573.0,695.0,653.0,...,56378.0,56367.0,56388.0,56408.0,56394.0,56378.0,125.912476,55.142212,0,1.0


In [46]:
df.describe()

,ecg_0,ecg_1,ecg_2,ecg_3,ecg_4,ecg_5,ecg_6,ecg_7,ecg_8,ecg_9,...,ppg_94,ppg_95,ppg_96,ppg_97,ppg_98,ppg_99,SBP,DBP,subject,trial
count,16047.000000,16047.000000,16047.000000,16047.000000,16047.000000,16047.000000,16047.000000,16047.000000,16047.000000,16047.000000,...,16047.00000,16047.000000,16047.000000,16047.000000,16047.000000,16047.000000,16047.000000,16047.000000,16047.000000,16047.000000
mean,1066.188945,431.663613,-1184.575871,-2584.160404,-2717.941609,-1756.763881,-644.251387,52.182402,383.462890,527.773914,...,43437.17536,43437.930953,43438.393968,43438.538605,43438.402817,43438.024553,123.233116,65.425236,3.436219,2.504393
std,596.022072,535.347140,1113.145288,1764.257444,1902.970274,1624.898566,1101.846380,532.029296,249.209632,288.587199,...,18823.48963,18824.907741,18826.414014,18827.809178,18828.936537,18829.911345,17.885018,12.599646,2.226186,1.117698
min,-5336.000000,-3104.000000,-5561.000000,-8135.000000,-8159.000000,-7152.000000,-4674.000000,-2434.000000,-1592.000000,-1497.000000,...,19505.00000,19481.000000,19474.000000,19463.000000,19443.000000,19433.000000,72.106934,27.767944,0.000000,1.000000
25%,641.000000,143.000000,-1963.500000,-3912.500000,-4168.500000,-2798.500000,-1271.500000,-216.000000,226.000000,323.000000,...,27992.50000,27994.500000,27999.500000,27995.000000,27992.500000,27983.000000,111.749268,56.675720,2.000000,2.000000
50%,780.000000,418.000000,-931.000000,-2302.000000,-2233.000000,-1286.000000,-169.000000,157.000000,344.000000,491.000000,...,35665.00000,35665.000000,35669.000000,35671.000000,35664.000000,35662.000000,123.074341,64.022827,3.000000,3.000000
75%,1637.500000,656.000000,-244.000000,-1088.500000,-1337.500000,-460.500000,129.000000,391.000000,499.000000,716.000000,...,59455.50000,59460.000000,59467.000000,59476.000000,59479.000000,59481.000000,133.680725,73.104858,5.000000,4.000000
max,4394.000000,3014.000000,1375.000000,946.000000,838.000000,1112.000000,2779.000000,2688.000000,2597.000000,2754.000000,...,83872.00000,83911.000000,83950.000000,83938.000000,83925.000000,83961.000000,191.830444,123.944092,7.000000,4.000000


In [47]:
# x y split
feature_cols = df.columns[:300]
target_cols = ['SBP', 'DBP']
groups = df['subject'].values

X = df[feature_cols].values
y = df[target_cols].values

In [55]:
groups.shape

(16047,)

In [48]:
# apply MinMaxScaler
scaler_x = MinMaxScaler()
X_scaled = scaler_x.fit_transform(X)
X = X_scaled

In [49]:
# reshape to (samples, timesteps, features)
X = X.reshape((-1, 3, 100))
X = np.transpose(X, (0, 2, 1))
X.shape, y.shape

((16047, 100, 3), (16047, 2))

In [50]:
logo = LeaveOneGroupOut()

In [ ]:
for train_idx, test_idx in logo.split(X, y, groups=groups):
    X_train, X_val = X[train_idx], X[test_idx]
    y_train, y_val = y[train_idx], y[test_idx]
    print(f'Train shape: {X_train.shape}, Test shape: {X_val.shape}')

    # define the model
    model = Sequential([
        Input(shape=(100, 3)),
        LSTM(64, activation='tanh', return_sequences=True),
        LSTM(32, activation='tanh'),
        Dense(16, activation='relu'),
        Dense(2)
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['mae', tf.keras.metrics.MeanAbsolutePercentageError()])
    print(model.summary())
    
    # early stopping callback
    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 5,
        restore_best_weights = True
    )
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=1,
        batch_size=32,
        callbacks=[early_stop]
    )

    loss, mae, mape = model.evaluate(X_val, y_val)
    print(f'Test Loss: {loss:.4f} - Test MAE: {mae:.4f} - Test MAPE: {mape:.4f}')


Train shape: (14243, 100, 3), Test shape: (1804, 100, 3)


Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_38 (LSTM)                  │ (None, 100, 64)        │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_39 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,386 (118.70 KB)

 Trainable params: 30,386 (118.70 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_52 (LSTM)                  │ (None, 100, 64)        │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_53 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,386 (118.70 KB)

 Trainable params: 30,386 (118.70 KB)

 Non-trainable params: 0 (0.00 B)

None
441/441 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/step - loss: 5913.9277 - mae: 66.6359 - mean_absolute_percentage_error: 66.2090 - val_loss: 1563.4960 - val_mae: 31.5606 - val_mean_absolute_percentage_error: 32.2448
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1563.4960 - mae: 31.5606 - mean_absolute_percentage_error: 32.2448
Test Loss: 1563.4960 - Test MAE: 31.5606 - Test MAPE: 32.2448
